# 神经网络的学习
The Study of Neural Network

## 4.1 从数据中学习
Learn from Data
- 线性可分问题是可解的
- 非线性可分问题则无法通过(自动)学习来解决

### 4.1.1 数据驱动
Driven by Data
- 数据是机器学习的核心
- 在机器学习领域中，常用的特征量包括SIFT，SURF，HOG等， 分类器有SVM， KNN等
- 机器学习的方法中，由机器从收集到的数据中找出规律性

针对机器学习任务的方法：
1. 人想到的算法 -> 答案
2. 人想到的特征量(SIFT,HOG) -> 机器学习(SVM,KNN) -> 答案
3. 神经网络(Deep Learning) -> 答案

深度学习有时也称为端到端机器学习(end-to-end machine learning)。
神经网络的优点是对所有的问题都可以用同样的流程来解决。

### 4.1.2 训练数据和测试数据
- 机器学习中，一般将数据分为训练数据和训练数据两部分逆行学习和实验
- 训练数据进行学习，寻找最优的参数
- 测试数据评价训练得到的模型的实际能力
- 指对某个数据集过度拟合的状态称为过拟合(overfitting)

## 4.2 损失函数
Loss Function
神经网络以某个指标为线索寻找最优权重参数，神经网络的学习中所用的指标称为损失函数(loss function)，可以为任意函数，但一般用均方误差和交叉熵误差等。

损失函数是表示神经网络性能的“恶劣程度”的指标，即当前的神经网络对监督数据(训练数据)在多大程度上不拟合，在多大程度上不一致。

### 4.2.1 均方差

In [1]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

### 4.2.2 交叉熵误差

In [2]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta)) # 避免log(0)变负无限大

### 4.2.3 mini-batch 学习
前面介绍的损失函数的额例子中考虑的都是针对单个数据的损失函数。
如果要求所有训练数据的损失函数的综合，以交叉熵误差为例子。

In [3]:
# we directly import dataset from Keras instead of loading the local dataset.
from keras.datasets import mnist
import numpy as np
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [4]:
x_train.shape

(60000, 28, 28)

In [5]:
x_train_res = x_train.reshape([60000, 28*28])

In [6]:
np.max(x_train_res)

255

In [7]:
np.min(x_train_res)

0

In [8]:
x_train = x_train_res/255
x_train.max()

1.0

In [9]:
x_test.shape

(10000, 28, 28)

In [10]:
x_test_res = x_test.reshape([10000, 28*28])
x_test = x_test_res/255

In [11]:
from keras.utils import to_categorical
y_train_10 = to_categorical(y_train)
y_test_10 = to_categorical(y_test)

In [12]:
print(x_train.shape)
print(y_train_10.shape)

(60000, 784)
(60000, 10)


In [13]:
# 在训练数据中随机抽取10笔数据
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
y_batch = y_train[batch_mask]

In [14]:
np.random.choice(60000, 10)

array([42928, 30741, 35400, 45465, 38473, 50907, 26099, 33552, 17371,
       56371])

### mini-batch版交叉熵误差的实现

In [15]:
def single01_cross_entropy_error(y, t):
    """
    参数
    ----
    y: np.array, output from network
    t: np.array, label from training dataset 
    
    返回
    ----
    np.float32
    """
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+1e-7))/batch_size

In [16]:
def single09_cross_entropy_error(y, t):
    """
    参数
    ----
    y: np.int, output from network
    t: np.int, one-hot label from training dataset
    
    返回
    ----
    np.float32
    """
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t]+1e-7))/batch_size
    

## 4.3 数值微分

### 4.3.1 导数

In [17]:
# 导数只在1个元里求导可能会出现计算结果为0的情况
def numerical_diff(f, x):
    h = 1e-4 # 0.0001
    return (f(x+h)-f(x-h))/(2*h)

### 4.3.2 数值微分的例子

In [18]:
def function_1(x):
    return 0.01 * x**2 + 0.1*x

In [19]:
import numpy as np
import matplotlib.pylab as plt

x = np.arange(0.0, 20.0, 0.1)
y = function_1(x)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.plot(x,y)
plt.show()

<Figure size 640x480 with 1 Axes>

In [20]:
print(numerical_diff(function_1, 5))
print(numerical_diff(function_1, 10))

0.1999999999990898
0.2999999999986347


### 4.3.3 偏导数

In [21]:
def function_2(x):
#     return np.sum(x**2)
    return x[0]**2+x[1]**2

## 4.4 梯度

In [22]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x) # 生成和x形状相同的数组
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h)的计算
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h)的计算
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 还原值
    return grad

In [23]:
print(numerical_gradient(function_2, np.array([3.0, 4.0])))

[6. 8.]


### 4.4.1 梯度法
Gradient Method
- 使用梯度寻找函数最小值的方法就是梯度法
- 梯度表示的是各点处的函数减小最多的方向
- 无法保证梯度所指的方向就是函数的最小值或者真正应该前进的方向
- 在复杂的函数中，梯度指示的方向基本上都不是函数值的最小处
- 即便如此，也要在寻找尽可能小的值的任务中，要以梯度的信息为线索，决定前进的方向
- 寻找最小值的为梯度下降法gradient descent method, 寻找最大值的为gradient ascent method

学习率：
    学习率过大，会发散成一个很大的值
    学习率过小，基本上没怎么更新就结束了

In [24]:
def gradient_descent(f, init_x, lr=0.01, step_number=100):
    """
    参数
    ----
    f: function, 
    init_x: np.array, the array needed to be optimized by gradient method
    lr: float32, learning rate
    step_number: int, repeat time
    """
    x = init_x
    
    for i in range(step_number):
        grad = numerical_gradient(f, x)
        x -= lr*grad
    
    return x

In [25]:
init_x = np.array([-3.0, 4.0])
x = gradient_descent(f=function_2, init_x=init_x, lr=0.1, step_number=100)

In [26]:
x

array([-6.11110793e-10,  8.14814391e-10])

### 4.4.2 神经网络的梯度


In [27]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

In [28]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

In [29]:
net = simpleNet()
print(net.W)

[[-0.09603916  0.19456455 -0.06848168]
 [-0.43689465 -0.60376565  1.20269565]]


In [30]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[-0.45082868 -0.42665035  1.04133708]


In [31]:
np.argmax(p)

2

In [32]:
t = np.array([0, 0, 1])
net.loss(x, t)

0.3751938971745548

In [33]:
def f(W):
    return net.loss(x, t)
dW = numerical_gradient(f, net.W)
print(dW)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [34]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

IndexError: index 2 is out of bounds for axis 0 with size 2

## 4.5 学习算法的实现

前提：
神经网络存在合适的权重和偏执，调整权重和偏置以拟合训练数据的过程为“学习”

步骤1:(mini-batch)
从训练数据中随机选出一部分数据，这部分数九称为mini-batch，我们的目标是减少mini-batch的孙志函数的值

步骤2:(计算梯度)
为了减少mini-batch损失的值，需要求出各个权重参数的梯度，梯度表示损失函数的值减小最多的方向

步骤3:(更新参数)
将权重参数沿梯度方向进行微小更新

步骤4:(重复)
重复步骤1，2，3 

In [45]:
import sys, os
sys.path.append('./code/common/')
from functions import *
from gradient import numerical_gradient

In [46]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # initialize the weight
        self.params = {}
        self.params['w1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['w2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        w1, w2 = self.params['w1'], self.params['w2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        grads = {}
        grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        return grads

In [47]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['w1'].shape

(784, 100)

In [48]:
x = np.random.rand(100, 784)
y = net.predict(x)

In [49]:
grads = net.numerical_gradient(x, t)
grads['w1'].shape

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (100,) (3,) 

### 4.5.2 mini-batch的实现

In [50]:
x_train_res.shape

(60000, 784)

In [51]:
x_test_res.shape

(10000, 784)

In [53]:
y_test.shape

(10000,)

In [56]:
y_test_10.shape

(10000, 10)

In [58]:
train_loss_list = []

# hyperparameters
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # get mini-batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train_res[batch_mask]
    t_batch = y_train_10[batch_mask]
    
    # calculate gradient
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) 高速版
    
    # update params
    for key in ('w1', 'b1', 'w2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
        
    # record the process of learning
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

KeyboardInterrupt: 

### 4.5.3 基于测试数据的评价

In [ ]:
train_loss_list = []
train_acc_list = []
test_acc_list = []

# 平均每个epoch的重复次数
iter_per_epoch = max(train_size / batch_size, 1)

# hyperparameters
iters_num = 10000
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # gain mini-batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train_res[batch_mask]
    y_batch = y_train_10[batch_mask]
        
    # calculate gradient
    grad = network.numerical_gradient(x_batch, y_batch)
    
    # update parameter
    for key in ('w1', 'b1', 'w2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
    
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)
    
    # 计算每个epoch的识别精度
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train_res, y_train_10)
        test_acc = network.accuracy(x_test_res, y_test_10)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('train acc, test acc | '+str(train_acc)+','+str(test_acc))